In [44]:
import pandas as pd
import numpy as np

### Load GMAT percentile data 

(Copy and pasted from [MBA website](https://www.mba.com/exams/gmat/after-the-exam/gmat-scores-and-score-reports/what-gmat-percentile-ranking-means).)

In [45]:
df = pd.read_csv('misc-files/gmat.csv')[:-1]

In [46]:
# Convert to float
df.per = df.per.str.rstrip('%').astype(np.float)

In [47]:
df

,gmat,per
0,800.0,99.0
1,790.0,99.0
2,780.0,99.0
3,770.0,99.0
4,760.0,99.0
...,...,...
56,240.0,1.0
57,230.0,1.0
58,220.0,1.0
59,210.0,0.0


### Define the GRE to GMAT conversion

Multiple linear regression equation taken from the [ETS website](https://www.ets.org/s/gre/pdf/background_and_technical_information.pdf).

In [48]:
def reg_equation(verb, quant):
    return -2080.75 + 6.38*verb + 10.62*quant

### Get all possible GRE scores and their GMAT equivalents

In [49]:
scores = np.arange(130, 171)
V, Q = np.meshgrid(scores, scores)

In [50]:
GRE = V + Q
GMAT = reg_equation(V, Q)

In [51]:
GRE

array([[260, 261, 262, ..., 298, 299, 300],
       [261, 262, 263, ..., 299, 300, 301],
       [262, 263, 264, ..., 300, 301, 302],
       ...,
       [298, 299, 300, ..., 336, 337, 338],
       [299, 300, 301, ..., 337, 338, 339],
       [300, 301, 302, ..., 338, 339, 340]])

In [52]:
GMAT

array([[129.25, 135.63, 142.01, ..., 371.69, 378.07, 384.45],
       [139.87, 146.25, 152.63, ..., 382.31, 388.69, 395.07],
       [150.49, 156.87, 163.25, ..., 392.93, 399.31, 405.69],
       ...,
       [532.81, 539.19, 545.57, ..., 775.25, 781.63, 788.01],
       [543.43, 549.81, 556.19, ..., 785.87, 792.25, 798.63],
       [554.05, 560.43, 566.81, ..., 796.49, 802.87, 809.25]])

#### Get bins to put the gre scores into percentiles

In [53]:
gmatbins = df.gmat.to_numpy()[:-1] - 5
gmatbins

array([795., 785., 775., 765., 755., 745., 735., 725., 715., 705., 695.,
       685., 675., 665., 655., 645., 635., 625., 615., 605., 595., 585.,
       575., 565., 555., 545., 535., 525., 515., 505., 495., 485., 475.,
       465., 455., 445., 435., 425., 415., 405., 395., 385., 375., 365.,
       355., 345., 335., 325., 315., 305., 295., 285., 275., 265., 255.,
       245., 235., 225., 215., 205.])

In [54]:
indices = np.digitize(GMAT.ravel(), bins=gmatbins)
percentiles = df.per.to_numpy()[indices]
percentiles

array([ 0.,  0.,  0., ..., 99., 99., 99.])

In [55]:
dfall = pd.DataFrame(list(zip(V.ravel(), Q.ravel(), GRE.ravel(), percentiles)), 
            columns=['Verbal', 'Quantitative', 'Composite', 'GMAT Percentile'])


In [56]:
dfall.to_csv('gre_composite-all_possible.csv', index=False)

In [57]:
dfcomp = pd.DataFrame(np.unique(np.array(list(zip(GRE.ravel(), percentiles))), axis=0), 
            columns=['Composite', 'GMAT Percentile'])

#### Unfortunately, the same composite GRE score can have multiple percentile scores, and vice versa

This is why we will take the median percentile for each composite GRE score. 

In [58]:
print(dfcomp.groupby('Composite')['GMAT Percentile'].nunique().tolist())

[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 2, 2, 3, 3, 4, 4, 4, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 12, 13, 13, 14, 15, 15, 15, 16, 16, 17, 17, 17, 16, 16, 16, 15, 15, 15, 14, 14, 13, 13, 13, 12, 12, 12, 11, 10, 10, 9, 9, 8, 8, 8, 7, 7, 7, 6, 6, 6, 5, 5, 4, 2, 1, 1, 1, 1, 1, 1]


In [59]:
dfcomp.loc[dfcomp['Composite'] == 300]

,Composite,GMAT Percentile
250,300.0,8.0
251,300.0,9.0
252,300.0,10.0
253,300.0,12.0
254,300.0,13.0
255,300.0,14.0
256,300.0,15.0
257,300.0,17.0
258,300.0,19.0
259,300.0,21.0


In [60]:
dfcomp.groupby('GMAT Percentile')['Composite'].std()

GMAT Percentile
0.0     4.183300
1.0     3.316625
2.0     4.183300
3.0     4.183300
4.0     4.472136
5.0     4.760952
6.0     4.472136
7.0     4.760952
8.0     5.049752
9.0     4.760952
10.0    5.049752
12.0    5.049752
13.0    5.049752
14.0    5.049752
15.0    5.049752
17.0    5.049752
19.0    5.049752
21.0    5.049752
23.0    5.049752
25.0    5.049752
27.0    5.049752
30.0    5.049752
31.0    5.049752
35.0    5.049752
38.0    5.049752
41.0    5.049752
44.0    4.760952
47.0    4.472136
51.0    4.472136
54.0    4.183300
58.0    4.183300
62.0    3.894440
65.0    3.605551
67.0    3.605551
73.0    3.316625
77.0    3.316625
80.0    3.027650
82.0    3.027650
85.0    2.738613
88.0    2.449490
91.0    2.449490
94.0    2.160247
96.0    2.160247
97.0    1.581139
98.0    1.581139
99.0    2.738613
Name: Composite, dtype: float64

#### Get the median percentile for each composite score

In [61]:
dfcomp2 = pd.DataFrame(dfcomp.groupby('Composite'
            )['GMAT Percentile'].median()).reset_index()


In [62]:
dfcomp2.to_csv('gre_composite-median.csv', index=False)

In [63]:
print(dfcomp2.set_index('Composite').to_markdown())

|   Composite |   GMAT Percentile |
|------------:|------------------:|
|         260 |               0   |
|         261 |               0   |
|         262 |               0   |
|         263 |               0   |
|         264 |               0   |
|         265 |               0   |
|         266 |               0   |
|         267 |               0   |
|         268 |               0   |
|         269 |               0.5 |
|         270 |               0.5 |
|         271 |               0.5 |
|         272 |               1   |
|         273 |               1   |
|         274 |               1.5 |
|         275 |               1.5 |
|         276 |               2   |
|         277 |               2   |
|         278 |               2.5 |
|         279 |               2.5 |
|         280 |               3.5 |
|         281 |               3.5 |
|         282 |               4   |
|         283 |               4.5 |
|         284 |               5   |
|         285 |             

In [65]:
print(dfall.set_index('Verbal').to_markdown())

|   Verbal |   Quantitative |   Composite |   GMAT Percentile |
|---------:|---------------:|------------:|------------------:|
|      130 |            130 |         260 |                 0 |
|      131 |            130 |         261 |                 0 |
|      132 |            130 |         262 |                 0 |
|      133 |            130 |         263 |                 0 |
|      134 |            130 |         264 |                 0 |
|      135 |            130 |         265 |                 0 |
|      136 |            130 |         266 |                 0 |
|      137 |            130 |         267 |                 0 |
|      138 |            130 |         268 |                 0 |
|      139 |            130 |         269 |                 0 |
|      140 |            130 |         270 |                 0 |
|      141 |            130 |         271 |                 0 |
|      142 |            130 |         272 |                 0 |
|      143 |            130 |         27